<a href="https://colab.research.google.com/github/Zain-Pirzada/Diabetic-Reinopathy-Kaggle-Competition/blob/main/Model_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf

In [7]:
from keras.applications import InceptionResNetV2
from keras.layers.core import Lambda
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_v3 import InceptionV3 
from keras.applications.densenet import DenseNet121
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
# from keras.utils.multi_gpu_utils import multi_gpu_model
from matplotlib import pyplot as plt
from keras.models import load_model
import keras.backend as K
import os
import tensorflow as tf


import numpy as np
import pandas as pd
from PIL import Image
import cv2 

import sys
import os.path

In [8]:

def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A

In [9]:
def Category_attention_block(inputs,classes,k):
    shape=K.int_shape(inputs)
    F=Conv2D(k*classes,1, padding='same') (inputs)
    F=BatchNormalization() (F)
    F1=Activation('relu') (F)
    
    F2=F1
    x=GlobalMaxPool2D()(F2)
    
    x=Reshape((classes,k)) (x)
    S=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    
    x=Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    x=Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (x)
    
    semantic=Multiply()([inputs,M])
    return semantic


In [10]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points    
   

In [11]:
def plotmodel(history,name):
    
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1) 
    
    plt.figure(1)                  
    plt.plot(epochs,smooth_curve(acc))
    plt.plot(epochs,smooth_curve(val_acc))
    plt.ylabel('acc')
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    plt.savefig('acc_'+name+'.png')
    
    plt.figure(2)
    plt.plot(epochs,smooth_curve(loss))
    plt.plot(epochs,smooth_curve(val_loss))
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss'], loc='upper right')
    plt.savefig('loss_'+name+'.png')

In [12]:
def label_smooth(y_true, y_pred):
    y_true=((1-0.1)*y_true+0.05)
    return K.categorical_crossentropy(y_true, y_pred) 
 

In [63]:
def get_base_model(model_name,image_size):
    if model_name =='vgg16':
        base_model=VGG16              (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='resnet50':
        base_model=ResNet50           (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='xception':
        base_model=Xception           (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='densenet121':
        base_model=DenseNet121       (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='mobilenet0.75':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=0.75,input_shape=(image_size,image_size,3))
    if model_name =='mobilenet1.0':
        base_model=MobileNet         (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='mobilenetv2':
        base_model=MobileNetV2      (include_top=False,weights='imagenet',alpha=1.0,input_shape=(image_size,image_size,3))
    if model_name =='inceptionv3':   
        base_model=InceptionV3       (include_top=False,weights='imagenet',input_shape=(image_size,image_size,3))
    if model_name =='inceptionv2':
        base_model=InceptionResNetV2 (include_top=False, weights='imagenet',input_shape=(image_size,image_size,3))
      
    return base_model

    

NameError: ignored

In [14]:
def clahe(cliplimit,gridsize,path):
    for img in os.listdir(path):
        imgarray = Image.open(os.path.join(path, img))
    lab = cv2.cvtColor(img_array, cv2.COLOR_BGR2Lab)

        # -----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)

        # -----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=cliplimit, tileGridSize=(gridsize, gridsize))
    cl = clahe.apply(l)

        # -----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl, a, b))

        # -----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

In [18]:
pip install -q tensorflow-model-optimization

In [70]:
from tensorflow_model_optimization.sparsity import keras as sparsity


In [65]:
def train_model(model,dataset,image_size,batch_size,save_name,lr1,lr2,Epochs1,Epochs2):
    
    #dataParam={'messidor': [957,243,2,'./data/messidor/train','./data/messidor/test'],
              #  'kaggle': [30000,5126,5,'./data/kaggle/train','./data/kaggle/valid'],
              #  'DDR':   [9851,2503,5,'./data/DDR/train','./data/DDR/valid']} #6119
    dataParam = {'DR': [1000,1000,5,'//content/drive/MyDrive/CABnet-master/data/train','//content/drive/MyDrive/CABnet-master/data/val']}

    train_num,valid_num,classes,train_dir,test_dir = dataParam[dataset]
    
    train=ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=90)          
    valid = ImageDataGenerator()
    train_data=train.flow_from_directory(train_dir,
                                         target_size=(image_size,image_size),
                                         shuffle = True,
                                         batch_size=batch_size)
    valid_data=valid.flow_from_directory(test_dir,
                                         target_size=(image_size,image_size),
                                         shuffle = False,
                                         batch_size=batch_size)

    lr_decay=ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=1)
    #save_model=ModelCheckpoint('new/'+save_name+'{epoch:02d}.h5', monitor='val_loss',period=10)
    # Define the pruning parameters
    pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                  final_sparsity=0.90,
                                                  begin_step=0,
                                                  end_step=5,
                                                  frequency=100)
        }


    
    for layer in base_model.layers:
        layer.trainable = False   

    #  Apply pruning to the model

    pruned_model.compile(optimizer=Adam(lr=lr1,decay=0.00001),loss=loss_fun,metrics=['acc'])
    pruned_model.fit_generator(train_data,
                        steps_per_epoch=train_num/batch_size,
                        validation_data=valid_data,
                        validation_steps=valid_num/batch_size,
                        epochs=Epochs1, 
                        workers=2,
                        callbacks=[lr_decay])   
    
    for layer in base_model.layers:
        layer.trainable = True
    
    
    pruned_model.compile(optimizer=Adam(lr=lr2,decay=0.00001),loss=loss_fun,metrics=['acc'])
    pruned_model.fit(train_data,
                        steps_per_epoch=train_num/batch_size,
                        validation_data=valid_data,
                        validation_steps=valid_num/batch_size,
                        epochs=Epochs2,
                        workers=2,
                        callbacks=[lr_decay])
    
    
    
    


In [66]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"    
loss_fun= 'categorical_crossentropy'  
gpu_num=1
k=5
lr1=0.005
lr2=0.0001
batch_size= 32
image_size=224
classes=5

base_model=get_base_model('vgg16',image_size)  
base_in=base_model.input
base_out=base_model.output

x=Global_attention_block(base_out)
base_out=Category_attention_block(x,classes,k)

x=GlobalAveragePooling2D()(base_out)
out=Dense(classes,activation='softmax')(x)

if gpu_num>1:
    model=Model(base_model.input,out)
    #model.summary()
    parallel_model = multi_gpu_model(model, gpus=gpu_num)
    parallel_model.summary()
else:
    parallel_model=Model(base_model.input,out)
    parallel_model.summary()


Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_14[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [67]:
history=train_model(parallel_model,'DR',image_size,batch_size,'vgg16',lr1,lr2,1,2)



Found 21073 images belonging to 5 classes.
Found 7023 images belonging to 5 classes.


NameError: ignored

In [ ]:
plotmodel(history,'vgg16')

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import load_model
# from keras import backend as K
# from keras.preprocessing import image
# import os
# import numpy as np

def label_smooth(y_true, y_pred):

    y_true=((1-0.1)*y_true+0.05)
    return K.categorical_crossentropy(y_true, y_pred)
    

def weight_kappa(result,test_num):
    weight=np.zeros((5,5),dtype='float')
    for i in range(5):
        for j in range(5):
            weight[i,j]=(i-j)*(i-j)/16
    fenzi=0
    for i in range(5):
        for j in range(5):
            fenzi=fenzi+result[i,j]*weight[i,j]
    fenmu=0
    for i in range(5):
        for j in range(5):
            fenmu=fenmu+weight[i,j]*result[:,j].sum()*result[i,:].sum()

    weght_kappa=1-(fenzi/(fenmu/test_num))
    return  weght_kappa

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
image_size=512
batch_size=64
model_name='inceptionv370.h5'
test_dir=''  
custom_test=True


if custom_test== False:   
    model=load_model('new/'+model_name)
else:
    model=load_model('new/'+model_name,custom_objects={'label_smooth': label_smooth})

test_num=0
result=np.zeros((5,5),dtype=int)
recall=np.zeros((1,5),dtype=float)
for i in range(5):
        datadirs=test_dir+str(i)+'/'
        filenames=os.listdir(datadirs)
        num=len(filenames)
        test_num=test_num+num
        valid = ImageDataGenerator()
        valid_data=valid.flow_from_directory(directory=test_dir,target_size=(image_size,image_size),
                                             batch_size=batch_size,class_mode=None,classes=str(i))
        predict=model.predict_generator(valid_data,steps=num/batch_size,verbose=1,workers=1)
        predict=np.argmax(predict,axis=-1)
        for j in range(5):
            result[i,j]=np.sum(predict==j)

right=result[0,0]+result[1,1]+result[2,2]+result[3,3]+result[4,4]
print('Acc:',right/test_num)

w_kappa=weight_kappa(result,test_num)
print('w_kappa:',w_kappa)


In [17]:
pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 6.3 MB/s eta 0:00:00


## Post Training Quantization

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

# Print TensorFlow info
print('TensorFlow version: '+tf.__version__)
print('Keras version: '+tf.keras.__version__)
print('TensorFlow Model Optimization version: '+tfmot.__version__)

TensorFlow version: 2.11.0
Keras version: 2.11.0
TensorFlow Model Optimization version: 0.7.3


In [ ]:
#Without Quantization
tflite_model = tf.keras.models.load_model("/content/drive/MyDrive/CABnet-master/vgg1621.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
tflite_save = converter.convert()

In [ ]:
len(tflite_save)

61036660

In [ ]:
#With Quantization
tflite_model = tf.keras.models.load_model("/content/drive/MyDrive/CABnet-master/vgg1621.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_post_quant_model = converter.convert()


In [ ]:
len(tflite_post_quant_model)

15350064

In [ ]:
with open("tflite_save.tflite", "wb") as f:
    f.write(tflite_save)

In [ ]:
with open("tflite_quant_model.tflite", "wb") as f:
    f.write(tflite_post_quant_model)

## Full Integer Quantization:

In [ ]:

# Load the pre-trained model
tflite_model = tf.keras.models.load_model("/content/drive/MyDrive/CABnet-master/vgg1621.h5")


# Create a full integer quantization object
#quantize_provider = tfmot.quantization.keras.quantize_annotate_layer(to_annotate, quantize_config=None)
#quantize_model = quantize_provider(tflite_model)
quantize_model = tflite_model


# Define the quantization parameters
converter = tf.lite.TFLiteConverter.from_keras_model(quantize_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define the representative dataset
def representative_dataset_gen():
    for i in range(100):
        yield [x_train[i:i+1]]
        

converter.representative_dataset_gen = representative_dataset_gen()


# Convert the model to TFLite format
tflite_full_integer_model = converter.convert()

# Save the TFLite model to disk
with open("model.tflite", "wb") as f:
    f.write(tflite_full_integer_model)



## Quantization Aware Training

In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(base_model)

# `quantize_model` requires a recompile.

q_aware_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
q_aware_model.summary()



Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 quantize_layer (QuantizeLay  (None, 224, 224, 3)      3         
 er)                                                             
                                                                 
 quant_block1_conv1 (Quantiz  (None, 224, 224, 64)     1923      
 eWrapperV2)                                                     
                                                                 
 quant_block1_conv2 (Quantiz  (None, 224, 224, 64)     37059     
 eWrapperV2)                                                     
                                                                 
 quant_block1_pool (Quantize  (None, 112, 112, 64)     1         
 WrapperV2)                                                  

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Convert the model to a quantization aware model
quantize_model = tf.keras.models.clone_model(base_model)
quantize_model = tfmot.quantization.keras.quantize_model(quantize_model)

# Convert the quantization aware model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the TensorFlow Lite model to disk
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)




In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow_model_optimization.sparsity import keras as sparsity


# Define the paths to the training and validation data
train_dir = '/content/drive/MyDrive/CABnet-master/data/train'
val_dir = '/content/drive/MyDrive/CABnet-master/data/val'

# Define the input shape of the model
input_shape = (224, 224, 3)

# Create a data generator for the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical'
)

# Create a data generator for the validation data
val_datagen = ImageDataGenerator(rescale=1./255)
val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical'
)

# Load the Keras model from the .h5 file
#keras_model_path = '/path/to/my_keras_model.h5'
#keras_model = tf.keras.models.load_model(base_model)

# Define the pruning parameters
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                  final_sparsity=0.90,
                                                  begin_step=0,
                                                  end_step=5,
                                                  frequency=100)
}

# Apply pruning to the model
pruned_model = sparsity.prune_low_magnitude(base_model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the pruned model
num_epochs = 10
pruned_model.fit(train_data, epochs=num_epochs, validation_data=val_data)

# Strip the pruning parameters from the pruned model
stripped_pruned_model = sparsity.strip_pruning(pruned_model)

# Save the pruned model to a .h5 file
tf.keras.models.save_model(stripped_pruned_model, 'my_pruned_model.h5', include_optimizer=False)


Found 21073 images belonging to 5 classes.
Found 7023 images belonging to 5 classes.
Epoch 1/10


ValueError: ignored

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
q_aware_model.fit(X_train, y_train, epochs=1)



NameError: name 'X_train' is not defined

In [ ]:
q_aware_model.evaluate(X_test, y_test)


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_qaware_model = converter.convert()

In [ ]:
len(tflite_qaware_model)

In [ ]:
with open("tflite_qaware_model.tflite", 'wb') as f:
    f.write(tflite_qaware_model)